In [16]:
import re
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Alphabet import generic_dna, generic_protein
from Bio.SeqRecord import SeqRecord


class Gene:
    def __init__(self, raw):
        self.raw = raw
        self.parts = raw.split('\n')
        self.name = re.search('pg.scf.\d+.g\d+', self.parts[0]).group(0)
        self.scaffold = re.search('pg.scf.\d+', self.parts[0]).group(0)
        self.transcript = re.search('\[[A-Z]+(\n# [A-Z]+)*\]', raw, re.MULTILINE).group(0)[1:-1].replace("\n# ", "")
        self.features = list(map(lambda f: Feature(f), filter(lambda l: not l.startswith('#'), self.parts)))
        
    def getFeatures(self, name):
        return list(filter(lambda f: f.feature == name, self.features))
    
    def fill(self, fasta_dict):
        for f in self.features:
            a1 = int(f.start) -1
            a2 = int(f.end) -1
            b1 = b2 = 0
            if f.feature == 'gene':
                b1 = a1
                b2 = a2
                a1 = max(0, a1 - 5000)
                a2 = min(len(fasta_dict[self.scaffold].seq) - 1, a2 + 5000)
                c1, c2 = b1 - a1, a2 - b2
                if c1 - 5000 != 0 or c2 - 5000 != 0:
                    print('%s # %d => %d (%d) : %d => %d : (%d)' % (self.name, b1, a1, c1, b2, a2, c2))
            f.seq = fasta_dict[self.scaffold][a1:a2]
        self.getFeatures('gene')[0]
        self.seq = self.getFeatures('gene')[0].getSeq()
    
    def getFeatureTypes(self, ):
        return list(set(map(lambda f: f.feature, self.features)))

        
class Feature:
    def __init__(self, raw):
        self.raw = raw
        self.parts = raw.split("\t")
        self.sequence = self.parts[0] #  1) The name of the sequence where the feature is located.
        self.source = self.parts[1] #    2) Keyword identifying the source of the feature, like a program (e.g. Augustus or RepeatMasker) or an organization (like TAIR).
        self.feature = self.parts[2]  #  3) The feature type name, like "gene" or "exon". In a well structured GFF file, all the children features always follow their parents in a single block (so all exons of a transcript are put after their parent "transcript" feature line and before any other parent transcript line). In GFF3, all features and their relationships should be compatible with the standards released by the Sequence Ontology Project.
        self.start = self.parts[3] #     4) Genomic start of the feature, with a 1-base offset. This is in contrast with other 0-offset half-open sequence formats, like BED files.
        self.end = self.parts[4] #       5) Genomic end of the feature, with a 1-base offset. This is the same end coordinate as it is in 0-offset half-open sequence formats, like BED files.[citation needed]
        self.score = self.parts[5] #     6) Numeric value that generally indicates the confidence of the source on the annotated feature. A value of "." (a dot) is used to define a null value.
        self.strand = self.parts[6] #    7) Single character that indicates the Sense (molecular biology) strand of the feature; it can assume the values of "+" (positive, or 5'->3'), "-", (negative, or 3'->5'), "." (undetermined).
        self.phase = self.parts[7] #     8) phase of CDS features; it can be either one of 0, 1, 2 (for CDS features) or "." (for everything else). See the section below for a detailed explanation.
        self.Attributes = self.parts[8] #9)All the other information pertaining to this feature. The format, structure and content of this field is the one which varies the most between the three competing file formats. 
    
    def getSeq(self):
        return str(self.seq.seq) if hasattr(self, 'seq') else None

        
class GFF:
    def __init__(self, file, fasta=None):
        ## Before use convert gff from Augustus to generic gff
        ## https://github.com/jorvis/biocode/blob/master/gff/convert_augustus_to_gff3.py
        self.file = file
        regex = r"# start gene pg.scf.\d+.g\d+\n# protein sequence = \[[A-Z]+(\n# [A-Z]+)*\]\n# end gene pg.scf.\d+.g\d+\n((.*\t){8}\S*\n)+###.*"
        self.genes = []
        self.features = set()
        with open(file, 'r') as f:
            if (not fasta is None):
                fasta_dict = SeqIO.to_dict(SeqIO.parse(fasta, "fasta"))
            raw = "".join(f.readlines())
            raw = (raw + "###") if raw.endswith('\n') else (raw + "\n###")
            for match in re.finditer(regex, raw, re.MULTILINE):
                gene = Gene(raw[int(match.start()):int(match.end())])
                if (not fasta is None):
                    gene.fill(fasta_dict)
                self.genes.append(gene)
                self.features = self.features.union(gene.getFeatureTypes())
            print('%d genes importados' % len(self.genes))
            print('features importados %s' % self.features)
    
    def asBioP(self, i, s, t=generic_dna):
        return SeqRecord(Seq(s, generic_protein), id=i, name=i, description=i)

    def exportTranscripts(self, file):
        SeqIO.write(list(map(lambda g: self.asBioP(g.name, g.transcript, generic_protein), self.genes)), file, "fasta")
    
    def exportCDS(self, file):   
        cdss = []
        for gene in self.genes:
            fs =  gene.getFeatures('CDS')
            for i in range(0, len(fs)):
                cdss.append(self.asBioP(gene.name + '.cds.' + str(i+1), fs[i].getSeq()))
        SeqIO.write(cdss, file, "fasta")
                    
#   def exportCDS(self, file):   
        cdss = []
        for gene in self.genes:
            fs =  gene.getFeatures('CDS')
            for i in range(0, len(fs)):
                cdss.append(self.asBioP(gene.name + '.cds.' + str(i+1), fs[i].getSeq()))
        SeqIO.write(cdss, file, "fasta")
            
    def exportGenes(self, file):
        SeqIO.write(list(map(lambda g: self.asBioP(g.name, g.seq), self.genes)), file, "fasta")
    
    def exportFastas(self, path_prefix=None, transcripts=True, cds=True, genes=True):
        
        prefix = re.search('(\S+\/)*\w+', self.file).group(0) if path_prefix is None else path_prefix
        prefix = (prefix + self.file.split('/')[-1].split('.')[0]) if prefix.endswith('/') else prefix
        
        if transcripts:
            self.exportTranscripts(prefix + ".proteins.fa")
        if cds:
            self.exportCDS(prefix + ".cds.fa")
        if genes:
            self.exportGenes(prefix + ".genes.fa")
    
#     def stats(self):
#         numero_de_pares_de_base
#         numero_de_scaffoolds
#         numero_de_genes
#         numero_de_proteinas
#         numero_de_cds
#         numero_de_exon
#         numero_de_intron
#         numero_de_mra
#         numero_de_genes_codificador_de_proteina
#         numero_de_genes_nao_codificador_de_proteina
#         densidade_de_gene
#         Gene density in  A: ($tot_genes_a_pc + $tot_genes_a_nc)/($tot_bases_a/1000) 
#                 ." per kb\n" if $tot_bases_a;

#         numero_de_genes_em_sobreposicao

#         ave_gene_length   
#         med_gene_length    
#         ave_exons_per_mRNA 
#         med_exons_per_mRNA 
#         ave_exon_length    
#         med_exon_length    
#         ave_intron_length
#         med_intron_length 
        
        

In [17]:
fasta = "/home/mfernandes/Documentos/linux/pybio/genome.fa"

In [18]:
goiaba = GFF("/home/mfernandes/Documentos/linux/pybio/augustus2.rna.guava.gff3", fasta)

pg.scf.1.g1 # 139 => 0 (139) : 870 => 1370 : (5000)
pg.scf.49.g403 # 0 => 0 (0) : 4718 => 5218 : (5000)
pg.scf.121.g725 # 446 => 0 (446) : 2975 => 3475 : (5000)
pg.scf.145.g798 # 0 => 0 (0) : 596 => 1096 : (5000)
pg.scf.217.g1022 # 81 => 0 (81) : 2867 => 3367 : (5000)
pg.scf.289.g1178 # 3 => 0 (3) : 3550 => 4050 : (5000)
pg.scf.313.g1270 # 337298 => 336798 (5000) : 339978 => 339978 : (0)
pg.scf.385.g1364 # 374 => 0 (374) : 2516 => 3016 : (5000)
pg.scf.385.g1398 # 297411 => 296911 (5000) : 297776 => 298235 : (459)
pg.scf.505.g1551 # 0 => 0 (0) : 214 => 714 : (5000)
pg.scf.553.g1626 # 70 => 0 (70) : 942 => 1442 : (5000)
pg.scf.577.g1654 # 0 => 0 (0) : 892 => 1392 : (5000)
pg.scf.601.g1681 # 0 => 0 (0) : 5637 => 6137 : (5000)
pg.scf.697.g1793 # 0 => 0 (0) : 2662 => 3162 : (5000)
pg.scf.793.g1932 # 0 => 0 (0) : 312 => 812 : (5000)
pg.scf.793.g1994 # 156753 => 156253 (5000) : 157040 => 157279 : (239)
pg.scf.889.g2097 # 87 => 0 (87) : 3416 => 3916 : (5000)
pg.scf.937.g2137 # 0 => 0 (0) : 207

pg.scf.28.g332 # 1043025 => 1042525 (5000) : 1046799 => 1046855 : (56)
pg.scf.124.g757 # 581676 => 581176 (5000) : 581933 => 582248 : (315)
pg.scf.148.g758 # 191 => 0 (191) : 3292 => 3792 : (5000)
pg.scf.172.g824 # 0 => 0 (0) : 1962 => 2462 : (5000)
pg.scf.460.g1478 # 257912 => 257412 (5000) : 258943 => 259339 : (396)
pg.scf.604.g1707 # 0 => 0 (0) : 564 => 1064 : (5000)
pg.scf.676.g1864 # 182946 => 182446 (5000) : 185621 => 185621 : (0)
pg.scf.700.g1865 # 0 => 0 (0) : 371 => 871 : (5000)
pg.scf.724.g1894 # 0 => 0 (0) : 1352 => 1852 : (5000)
pg.scf.820.g1991 # 201 => 0 (201) : 3530 => 4030 : (5000)
pg.scf.916.g2068 # 155 => 0 (155) : 3561 => 4061 : (5000)
pg.scf.1036.g2151 # 0 => 0 (0) : 5320 => 5820 : (5000)
pg.scf.1060.g2162 # 343 => 0 (343) : 993 => 1493 : (5000)
pg.scf.1132.g2210 # 0 => 0 (0) : 7360 => 7860 : (5000)
pg.scf.1132.g2221 # 96066 => 95566 (5000) : 97487 => 97618 : (131)
pg.scf.1156.g2222 # 287 => 0 (287) : 4027 => 4527 : (5000)
pg.scf.1204.g2251 # 0 => 0 (0) : 1884 => 23

pg.scf.31.g196 # 0 => 0 (0) : 882 => 1382 : (5000)
pg.scf.151.g869 # 521027 => 520527 (5000) : 525614 => 525908 : (294)
pg.scf.175.g870 # 0 => 0 (0) : 1080 => 1580 : (5000)
pg.scf.247.g1065 # 0 => 0 (0) : 702 => 1202 : (5000)
pg.scf.247.g1117 # 397242 => 396742 (5000) : 397997 => 398133 : (136)
pg.scf.271.g1171 # 374644 => 374144 (5000) : 376496 => 376524 : (28)
pg.scf.295.g1220 # 355084 => 354584 (5000) : 355770 => 355860 : (90)
pg.scf.343.g1256 # 0 => 0 (0) : 2261 => 2761 : (5000)
pg.scf.439.g1402 # 38 => 0 (38) : 274 => 774 : (5000)
pg.scf.463.g1473 # 254862 => 254362 (5000) : 258250 => 258747 : (497)
pg.scf.487.g1474 # 0 => 0 (0) : 8249 => 8749 : (5000)
pg.scf.535.g1539 # 0 => 0 (0) : 10329 => 10829 : (5000)
pg.scf.631.g1675 # 147 => 0 (147) : 437 => 937 : (5000)
pg.scf.679.g1763 # 183421 => 182921 (5000) : 184944 => 185063 : (119)
pg.scf.703.g1784 # 176200 => 175700 (5000) : 177153 => 177240 : (87)
pg.scf.847.g1907 # 35 => 0 (35) : 676 => 1176 : (5000)
pg.scf.871.g1948 # 139474 =>

pg.scf.2697.g2533 # 10785 => 10285 (5000) : 11414 => 11689 : (275)
pg.scf.2745.g2537 # 8028 => 7528 (5000) : 9545 => 9545 : (0)
pg.scf.2793.g2539 # 0 => 0 (0) : 268 => 768 : (5000)
pg.scf.2817.g2540 # 2009 => 1509 (5000) : 5116 => 5402 : (286)
pg.scf.10.g1 # 0 => 0 (0) : 806 => 1306 : (5000)
pg.scf.178.g851 # 479729 => 479229 (5000) : 481190 => 481190 : (0)
pg.scf.250.g958 # 0 => 0 (0) : 4806 => 5306 : (5000)
pg.scf.274.g1086 # 372435 => 371935 (5000) : 372884 => 373240 : (356)
pg.scf.298.g1135 # 352412 => 351912 (5000) : 354062 => 354062 : (0)
pg.scf.394.g1319 # 284483 => 283983 (5000) : 289725 => 289825 : (100)
pg.scf.490.g1427 # 0 => 0 (0) : 260 => 760 : (5000)
pg.scf.490.g1469 # 246635 => 246135 (5000) : 247587 => 247859 : (272)
pg.scf.562.g1579 # 0 => 0 (0) : 1226 => 1726 : (5000)
pg.scf.562.g1613 # 213078 => 212578 (5000) : 213808 => 213907 : (99)
pg.scf.634.g1673 # 0 => 0 (0) : 857 => 1357 : (5000)
pg.scf.754.g1801 # 0 => 0 (0) : 1219 => 1719 : (5000)
pg.scf.802.g1857 # 154934 =

pg.scf.37.g163 # 0 => 0 (0) : 2071 => 2571 : (5000)
pg.scf.85.g479 # 693917 => 693417 (5000) : 694408 => 694662 : (254)
pg.scf.229.g804 # 0 => 0 (0) : 860 => 1360 : (5000)
pg.scf.277.g927 # 0 => 0 (0) : 667 => 1167 : (5000)
pg.scf.277.g968 # 370923 => 370423 (5000) : 371195 => 371254 : (59)
pg.scf.397.g1176 # 283690 => 283190 (5000) : 289098 => 289098 : (0)
pg.scf.493.g1278 # 0 => 0 (0) : 1108 => 1608 : (5000)
pg.scf.541.g1338 # 428 => 0 (428) : 1450 => 1950 : (5000)
pg.scf.565.g1368 # 0 => 0 (0) : 260 => 760 : (5000)
pg.scf.565.g1369 # 441 => 0 (441) : 1867 => 2367 : (5000)
pg.scf.589.g1424 # 205913 => 205413 (5000) : 206941 => 207023 : (82)
pg.scf.613.g1425 # 100 => 0 (100) : 1373 => 1873 : (5000)
pg.scf.613.g1450 # 200843 => 200343 (5000) : 201700 => 202015 : (315)
pg.scf.685.g1532 # 179038 => 178538 (5000) : 183015 => 183321 : (306)
pg.scf.709.g1533 # 114 => 0 (114) : 353 => 853 : (5000)
pg.scf.805.g1649 # 154206 => 153706 (5000) : 154936 => 155371 : (435)
pg.scf.901.g1731 # 129181

pg.scf.232.g905 # 0 => 0 (0) : 2817 => 3317 : (5000)
pg.scf.520.g1454 # 13 => 0 (13) : 630 => 1130 : (5000)
pg.scf.592.g1553 # 0 => 0 (0) : 2063 => 2563 : (5000)
pg.scf.640.g1609 # 211 => 0 (211) : 567 => 1067 : (5000)
pg.scf.640.g1630 # 193678 => 193178 (5000) : 194579 => 194588 : (9)
pg.scf.688.g1678 # 180228 => 179728 (5000) : 182402 => 182402 : (0)
pg.scf.784.g1739 # 304 => 0 (304) : 2667 => 3167 : (5000)
pg.scf.808.g1781 # 151660 => 151160 (5000) : 154656 => 154656 : (0)
pg.scf.832.g1782 # 0 => 0 (0) : 1019 => 1519 : (5000)
pg.scf.904.g1868 # 130531 => 130031 (5000) : 131364 => 131437 : (73)
pg.scf.928.g1869 # 79 => 0 (79) : 450 => 950 : (5000)
pg.scf.928.g1892 # 122471 => 121971 (5000) : 125744 => 125744 : (0)
pg.scf.1000.g1941 # 0 => 0 (0) : 8485 => 8985 : (5000)
pg.scf.1096.g1995 # 9 => 0 (9) : 2447 => 2947 : (5000)
pg.scf.1216.g2082 # 84083 => 83583 (5000) : 87199 => 87254 : (55)
pg.scf.1240.g2097 # 82338 => 81838 (5000) : 84641 => 84888 : (247)
pg.scf.1264.g2098 # 197 => 0 (1

pg.scf.67.g391 # 751537 => 751037 (5000) : 752677 => 753102 : (425)
pg.scf.91.g392 # 402 => 0 (402) : 4001 => 4501 : (5000)
pg.scf.163.g662 # 0 => 0 (0) : 1837 => 2337 : (5000)
pg.scf.163.g726 # 499389 => 498889 (5000) : 499604 => 499989 : (385)
pg.scf.307.g1032 # 0 => 0 (0) : 4412 => 4912 : (5000)
pg.scf.499.g1408 # 0 => 0 (0) : 2137 => 2637 : (5000)
pg.scf.523.g1440 # 367 => 0 (367) : 723 => 1223 : (5000)
pg.scf.571.g1516 # 0 => 0 (0) : 533 => 1033 : (5000)
pg.scf.619.g1568 # 0 => 0 (0) : 679 => 1179 : (5000)
pg.scf.619.g1604 # 198991 => 198491 (5000) : 200256 => 200666 : (410)
pg.scf.667.g1645 # 340 => 0 (340) : 1247 => 1747 : (5000)
pg.scf.691.g1724 # 180138 => 179638 (5000) : 181234 => 181611 : (377)
pg.scf.739.g1746 # 0 => 0 (0) : 1364 => 1864 : (5000)
pg.scf.907.g1904 # 0 => 0 (0) : 365 => 865 : (5000)
pg.scf.979.g1957 # 0 => 0 (0) : 181 => 681 : (5000)
pg.scf.1051.g2018 # 0 => 0 (0) : 357 => 857 : (5000)
pg.scf.1075.g2045 # 103905 => 103405 (5000) : 104584 => 104584 : (0)
pg.sc

pg.scf.94.g457 # 678386 => 677886 (5000) : 680329 => 680809 : (480)
pg.scf.166.g625 # 0 => 0 (0) : 271 => 771 : (5000)
pg.scf.262.g867 # 279 => 0 (279) : 1985 => 2485 : (5000)
pg.scf.286.g964 # 364322 => 363822 (5000) : 364667 => 364667 : (0)
pg.scf.502.g1360 # 240905 => 240405 (5000) : 241291 => 241766 : (475)
pg.scf.598.g1468 # 0 => 0 (0) : 2852 => 3352 : (5000)
pg.scf.646.g1520 # 0 => 0 (0) : 1206 => 1706 : (5000)
pg.scf.646.g1546 # 192162 => 191662 (5000) : 192848 => 192857 : (9)
pg.scf.766.g1639 # 0 => 0 (0) : 2947 => 3447 : (5000)
pg.scf.934.g1787 # 0 => 0 (0) : 2807 => 3307 : (5000)
pg.scf.1006.g1831 # 0 => 0 (0) : 510 => 1010 : (5000)
pg.scf.1006.g1853 # 109614 => 109114 (5000) : 113533 => 113681 : (148)
pg.scf.1030.g1854 # 0 => 0 (0) : 1404 => 1904 : (5000)
pg.scf.1078.g1882 # 0 => 0 (0) : 2375 => 2875 : (5000)
pg.scf.1078.g1901 # 103040 => 102540 (5000) : 103987 => 104225 : (238)
pg.scf.1198.g1945 # 431 => 0 (431) : 3711 => 4211 : (5000)
pg.scf.1222.g1954 # 0 => 0 (0) : 6689 

pg.scf.2376.g2477 # 15623 => 15123 (5000) : 21837 => 21971 : (134)
pg.scf.2400.g2478 # 0 => 0 (0) : 230 => 730 : (5000)
pg.scf.2544.g2505 # 16095 => 15595 (5000) : 16559 => 16846 : (287)
pg.scf.2640.g2513 # 12954 => 12454 (5000) : 13645 => 13645 : (0)
pg.scf.2664.g2518 # 12634 => 12134 (5000) : 12951 => 13001 : (50)
pg.scf.2736.g2523 # 0 => 0 (0) : 1551 => 2051 : (5000)
pg.scf.2832.g2526 # 2948 => 2448 (5000) : 4379 => 4379 : (0)
61089 genes importados
features importados {'mRNA', 'exon', 'gene', 'CDS'}


In [19]:
goiaba.exportGenes("/home/mfernandes/Documentos/linux/pybio/genes5000.fa")

In [15]:
goiaba.exportFastas()